In [7]:
import os
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [8]:
# Connect to a cluster
h2o.init()

H2O cluster uptime:,23 minutes 22 seconds 951 milliseconds
H2O cluster version:,3.8.3.2
H2O cluster name:,H2O_started_from_R_amy_dmi326
H2O cluster total nodes:,1
H2O cluster total free memory:,2.84 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


In [9]:
## Define data paths
base_path = os.path.abspath("../data")
flights_path = base_path + "/flights.csv"
weather_path = base_path + "/weather.csv"

In [10]:
## Ingest data
flights_hex = h2o.import_file(path = flights_path, destination_frame = "flights_hex")
weather_hex = h2o.import_file(path = weather_path, destination_frame = "weather_hex")


Parse Progress: [##################################################] 100%

Parse Progress: [##################################################] 100%


In [11]:
## Summary of the flights and weather dataset
flights_hex.show()
weather_hex.show()

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
1987,10,3,6,907,907,1511,1500,AA,38,nan,244,233,nan,11,0,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,YES,NO
1987,10,6,2,907,907,1507,1500,AA,38,nan,240,233,nan,7,0,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,YES,NO
1987,10,8,4,907,907,1451,1500,AA,38,nan,224,233,nan,-9,0,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,NO,NO
1987,10,9,5,906,907,1449,1500,AA,38,nan,223,233,nan,-11,-1,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,NO,NO
1987,10,10,6,906,907,1434,1500,AA,38,nan,208,233,nan,-26,-1,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,NO,NO
1987,10,11,7,907,907,1440,1500,AA,38,nan,213,233,nan,-20,0,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,NO,NO
1987,10,12,1,906,907,1456,1500,AA,38,nan,230,233,nan,-4,-1,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,NO,NO
1987,10,14,3,906,907,1511,1500,AA,38,nan,245,233,nan,11,-1,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,YES,NO
1987,10,15,4,906,907,1444,1500,AA,38,nan,218,233,nan,-16,-1,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,NO,NO
1987,10,16,5,907,907,1451,1500,AA,38,nan,224,233,nan,-9,0,SAN,ORD,1723,nan,nan,0,nan,0,0,0,0,0,0,NO,NO


ICAO,MaxTempF,MeanTempF,MinTempF,MaxDewPointF,MeanDewPointF,Min DewpointF,MaxHumidity,MeanHumidity,MinHumidity,MaxSeaLevelPressureIn,MeanSeaLevelPressureIn,MinSeaLevelPressureIn,MaxVisibilityMiles,MeanVisibilityMiles,MinVisibilityMiles,MaxWindSpeedMPH,MeanWindSpeedMPH,MaxGustSpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees,Dest,Year,Month,DayofMonth
KORD,33,32,30,32,29,24,96,90,79,30.14,30.03,29.95,10,3,1,9,5,nan,0.16,8,Fog-Snow,308,ORD,1987,1,1
KORD,36,33,30,33,31,29,100,90,82,30.18,30.02,29.94,8,6,2,10,7,nan,0.01,7,Fog-Snow,323,ORD,1987,1,2
KORD,39,33,27,30,29,26,100,87,65,30.34,30.28,30.19,9,7,5,12,5,nan,0,7,Fog,273,ORD,1987,1,3
KORD,37,28,19,31,26,19,100,89,65,30.42,30.38,30.32,8,4,0,8,4,nan,0,4,Fog,332,ORD,1987,1,4
KORD,39,32,25,30,27,24,100,81,62,30.39,30.27,30.07,10,8,3,13,7,nan,0,5,Fog,180,ORD,1987,1,5
KORD,46,40,34,38,31,26,86,73,62,30.04,29.8,29.61,15,10,10,23,14,nan,0,7,Rain,226,ORD,1987,1,6
KORD,37,35,32,31,28,26,82,75,70,30.27,30.16,29.93,15,11,7,14,10,nan,0,8,,309,ORD,1987,1,7
KORD,32,29,26,27,22,18,85,74,61,30.27,30.22,30.16,15,12,7,17,11,nan,0,7,Snow,268,ORD,1987,1,8
KORD,30,26,21,30,25,19,100,89,82,30.14,29.9,29.61,15,6,0,16,8,nan,0.46,6,Fog-Snow,147,ORD,1987,1,9
KORD,34,32,30,32,29,25,100,89,75,29.91,29.72,29.57,9,4,2,20,12,26,0.15,8,Fog-Snow,339,ORD,1987,1,10


In [ ]:
# Group flights by Year
flights_hex["IsArrDelayedNumeric"] = (flights_hex["IsArrDelayed"] == "YES").ifelse(1,0)
flights_hex["IsWeatherDelayedNumeric"] = (flights_hex["WeatherDelay"] > 0).ifelse(1,0)
flights_group = flights_hex.group_by("Year")
flights_count = flights_group.count().sum("IsArrDelayedNumeric").sum("IsWeatherDelayedNumeric").frame
flights_count.as_data_frame()

In [ ]:
## Filter flights before 2003
flights_hex = flights_hex[ flights_hex[ "Year"] >= 2003]
## Filter flights that is delayed but not delayed by weather
flights_hex = flights_hex[ (flights_hex["IsArrDelayed"] == "NO") | (flights_hex["WeatherDelay"] > 0)]

In [ ]:
flights_hex["CRSArrTime"]

In [ ]:
## Parameter Creation
hour1 = flights_hex["CRSArrTime"] // 100
mins1 = flights_hex["CRSArrTime"] % 100
arrTime = hour1*60+mins1
hour2 = flights_hex["CRSDepTime"] // 100
mins2 = flights_hex["CRSDepTime"] % 100
depTime = hour2*60+mins2

travelTime = (arrTime - depTime > 0).ifelse(arrTime - depTime, 0)
flights_hex[ "TravelTime"]  = travelTime

In [ ]:
flights_hex[ "TravelTime"]

In [ ]:
# Set predictor and response variables
myY = "IsArrDelayed"
myX = ["Year", "Month", "DayofMonth", "DayOfWeek", "UniqueCarrier", "Origin", "TravelTime"]

In [ ]:
# Create test/train split
split = flights_hex["Year"].runif()
train = flights_hex[split <= 0.75]
valid = flights_hex[split > 0.75]

In [ ]:

# GLM - Predict Delays
arr_delay_glm = H2OGeneralizedLinearEstimator(family="binomial",
                                              standardize=True, 
                                              alpha = 0.5, 
                                              lambda_search = True)
arr_delay_glm.train(x               =myX,
                    y               =myY,
                    training_frame  =train,
                    validation_frame=valid)

# GBM
arr_delay_gbm = H2OGradientBoostingEstimator(distribution   ="bernoulli",
                                             ntrees         =50)

arr_delay_gbm.train(x               =myX,
                    y               =myY,
                    training_frame  =train,
                    validation_frame=valid)


In [ ]:
print "GLM AUC TRAIN=", arr_delay_glm.auc(train = True),", AUC Valid=",arr_delay_glm.auc(valid = True)
print "GBM AUC TRAIN=", arr_delay_gbm.auc(train = True),", AUC Valid=",arr_delay_gbm.auc(valid = True)

In [ ]:
## Merge with weather data
merged_data = flights_hex.merge(weather_hex)

In [ ]:
# Create test/train split
split_weather = merged_data["Year"].runif()
train_weather = merged_data[split <= 0.75]
valid_weather = merged_data[split > 0.75]

In [ ]:
# Set predictor and response variables
myY = "IsArrDelayed"
myX = ["Year", "Month", "DayofMonth", "DayOfWeek", "UniqueCarrier", "Origin", "TravelTime"] + weather_hex.columns

In [ ]:
# GLM - Predict Delays
arr_delay_weather_glm = H2OGeneralizedLinearEstimator(family="binomial",
                                                      standardize=True, 
                                                      alpha = 0.5, 
                                                      lambda_search = True)
arr_delay_weather_glm.train(x               =myX,
                            y               =myY,
                            training_frame  =train_weather,
                            validation_frame=valid_weather)

# GBM
arr_delay_weather_gbm = H2OGradientBoostingEstimator(distribution   ="bernoulli",
                                                     ntrees         =50)

arr_delay_weather_gbm.train(x               =myX,
                            y               =myY,
                            training_frame  =train_weather,
                            validation_frame=valid_weather)



In [ ]:
print "GLM AUC TRAIN=", arr_delay_weather_glm.auc(train = True),", AUC Valid=",arr_delay_weather_glm.auc(valid = True)
print "GBM AUC TRAIN=", arr_delay_weather_gbm.auc(train = True),", AUC Valid=",arr_delay_weather_gbm.auc(valid = True)